In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR, ensure_dir
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/home/AD/tsainbur/anaconda3/envs/childes-paper/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [5]:
!ls /mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/mi/phonbank_longest_seqs-french.pickle

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/mi/phonbank_longest_seqs-french.pickle


In [6]:
MI_DF = pd.read_pickle("/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/data/mi/phonbank_longest_seqs-french.pickle")

In [7]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_phones,n_unique_phones,age,dataset,xml_loc,n_phones,phones_list
0,"[4.220272462086125, 3.827182912393157, 3.65988...","[0.03966478656415546, 0.03863028781460936, 0.0...","[3.5478525638018503, 3.549876481214623, 3.5626...","[0.038492261314211185, 0.03839394643764451, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2607,1,1.00,Lyon,/mnt/cube/tsainbur/Projects/github_repos/child...,949,mmliːʀmwibʀʀʀɡiliɡiliɡiltyliːepɔʀteːzøvøʒvødã...
1,"[4.003031163827437, 3.9261375225418274, 3.6549...","[0.06548629734066684, 0.06372777471464103, 0.0...","[3.416818373426647, 3.413034965699241, 3.42533...","[0.06352383582895753, 0.06343581412051416, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,1858,1,1.00,Paris,/mnt/cube/tsainbur/Projects/github_repos/child...,789,eːbɛvœgagatœ̃kakadœjøkɛkamtakadɛkagøgegakagagɛ...
2,"[4.123307160457033, 3.802053605125085, 3.66670...","[0.05585662525034512, 0.054689141875846704, 0....","[3.541364106943293, 3.5215856948513924, 3.5299...","[0.05445856150651234, 0.0542388669284434, 0.05...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2405,1,0.75,KernFrench,/mnt/cube/tsainbur/Projects/github_repos/child...,690,VVVVVnaVVVʙʙʙʙʙʙʙʙʙaːVbəaɟajaVblɛtCCCCCbaabaga...


In [8]:
MI_DF.age_cohort_low.unique()

array([0.5, 1. , 1.5, 2. , 2.5, 3. ])

In [9]:
age_cohorts = [[0.5, 1],[1, 1.5], [1.5, 2], [2, 2.5], [2.5, 3], [3, 20]]

In [10]:
MI_DF_list = []
for [ac_low, ac_high] in age_cohorts:
    MI_DF_list.append(MI_DF[MI_DF.age_cohort_low == ac_low][:10])
MI_DF = pd.concat(MI_DF_list)

In [11]:
len(MI_DF)

28

In [12]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [13]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    dist_mask = distances < 100
    distances = distances[dist_mask]
    sig = sig[dist_mask]
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan
    
    
    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

  0%|          | 0/28 [00:00<?, ?it/s]

/mnt/cube/tsainbur/Projects/github_repos/LRSO_paper_rev/LongRangeSequentialOrgPaper/childes_mi/information_theory/model_fitting.py:8: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(y_true) - np.log(y_model)) * (1 / (np.log(1 + x)))
/home/AD/tsainbur/anaconda3/envs/childes-paper/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [14]:
MI_DF.to_pickle(DATA_DIR / "mi" / "phonbank_longest_seqs-french-fitted.pickle")